In [1]:
from huggingface_hub import hf_hub_download

repo_id = "sayakpaul/ucf101-subset"
filename = "v_BasketballDunk_g14_c06.avi"
repo_type = "dataset"

kwargs = dict(repo_id=repo_id, repo_type=repo_type, filename=filename)
video_path = hf_hub_download(**kwargs)

In [3]:
import scenedetect
from scenedetect.detectors import ContentDetector, ThresholdDetector
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager

def detect_scenes(video_path, detector):
    # Create a video manager and scene manager
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()

    # Add the detector to the scene manager
    scene_manager.add_detector(detector)

    # Set downscale factor to improve processing speed (optional)
    video_manager.set_downscale_factor()

    # Start the video manager
    video_manager.start()

    # Detect scenes
    scene_manager.detect_scenes(frame_source=video_manager)

    # Obtain list of detected scenes
    return scene_manager.get_scene_list()

# Video path
# video_path = "path/to/your/video.mp4"

# Define three detectors with different thresholds
detector1 = ContentDetector(threshold=30)
detector2 = ContentDetector(threshold=15)
detector3 = ContentDetector(threshold=5)

# Detect scenes with each detector
scenes1 = detect_scenes(video_path, detector1)
scenes2 = detect_scenes(video_path, detector2)
scenes3 = detect_scenes(video_path, detector3)

# Combine results (this step may require custom logic based on your requirements)
combined_scenes = scenes1 + scenes2 + scenes3
combined_scenes = list(set(combined_scenes))  # Remove duplicates, if any

# Print scene list
for scene in combined_scenes:
    print(f"Scene: {scene[0].get_frames()}, {scene[1].get_frames()}")

VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.


Scene: 82, 97
Scene: 51, 66
Scene: 0, 96
Scene: 97, 122
Scene: 31, 51
Scene: 96, 125
Scene: 0, 15
Scene: 122, 125
Scene: 66, 82
Scene: 15, 31


In [4]:
import subprocess
import re

def get_keyframe_timestamps(video_path):
    command = ["ffprobe", "-select_streams", "v", "-show_frames", "-show_entries", "frame=pict_type", "-of", "csv", video_path]
    output = subprocess.check_output(command).decode()
    
    keyframe_timestamps = []
    for line in output.split('\n'):
        if 'pict_type=I' in line:
            frame_number = int(re.search(r'frame,([0-9]+)', line).group(1))
            keyframe_timestamps.append(frame_number)

    return keyframe_timestamps

In [5]:
def snap_to_keyframes(cuts, keyframes):
    snapped_cuts = []
    for start, end in cuts:
        start_keyframe = min(keyframes, key=lambda k: abs(k - start))
        end_keyframe = min(keyframes, key=lambda k: abs(k - end))
        snapped_cuts.append((start_keyframe, end_keyframe))
    return snapped_cuts

def extract_clips(video_path, cuts, frame_rate):
    for i, (start, end) in enumerate(cuts):
        start_time = start / frame_rate
        end_time = end / frame_rate
        duration = end_time - start_time
        output_file = f"clip_{i}.mp4"
        command = ["ffmpeg", "-ss", str(start_time), "-i", video_path, "-t", str(duration), "-c", "copy", output_file]
        subprocess.call(command)

In [6]:
def detect_cuts(video_path):
    scenes1 = detect_scenes(video_path, detector1)
    scenes2 = detect_scenes(video_path, detector2)
    scenes3 = detect_scenes(video_path, detector3)
    
    # Combine results (this step may require custom logic based on your requirements)
    combined_scenes = scenes1 + scenes2 + scenes3
    combined_scenes = list(set(combined_scenes))  # Remove duplicates, if any
    return combined_scenes

In [7]:
# video_path = "path/to/your/video.mp4"
frame_rate = 25  # Replace with the actual frame rate of your video

cuts = detect_cuts(video_path)
keyframes = get_keyframe_timestamps(video_path)
snapped_cuts = snap_to_keyframes(cuts, keyframes)
extract_clips(video_path, snapped_cuts, frame_rate)

VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
ffprobe version 4.2.7-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enab